# Galactic dust realization

**NOTE**: an executed version of this notebooks can be find [here](https://gist.github.com/javierhndev/b58920472e7906110585fb6cb6b92b74)

In this notebook we run PySM to generate a galactic dust map and analyze that map using the cl spectrum.

Then we check the impact of the following:
- Lower resolution map.
- From sky map to 2D cartesian.
- Cropped image.
- Save and read image file (using HDF to save wcs info).

In [ ]:
# pip install numpy==1.26.4 scipy==1.13.1 pixell==0.26.0
#%pip install pysm3[test] healpy

In [ ]:
import pysm3
from matplotlib import pyplot as plt
from matplotlib.image import imread
import numpy as np
import pixell
from pysm3 import units as u
import healpy as hp
from PIL import Image

## Define the resolution

In [ ]:
nside = 256  # higher for high resolution
healpix_reso = hp.nside2resol(nside) * u.radian
healpix_reso.to(u.arcmin)

In [ ]:
npix = hp.nside2npix(nside)
print(npix / 1e6, "Mpix")

## PySM simulation

In [ ]:
seed = 101

In [ ]:
dust = pysm3.ModifiedBlackBodyRealization(
    nside=nside,
    amplitude_modulation_temp_alm="dust_gnilc/raw/gnilc_dust_temperature_modulation_alms_lmax768.fits.gz",
    amplitude_modulation_pol_alm="dust_gnilc/raw/gnilc_dust_polarization_modulation_alms_lmax768.fits.gz",
    largescale_alm="dust_gnilc/raw/gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax1024_complex64.fits.gz",
    small_scale_cl="dust_gnilc/raw/gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz",
    largescale_alm_mbb_index="dust_gnilc/raw/gnilc_dust_largescale_template_beta_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_index="dust_gnilc/raw/gnilc_dust_small_scales_beta_cl_lmax16384_2023.06.06.fits.gz",
    largescale_alm_mbb_temperature="dust_gnilc/raw/gnilc_dust_largescale_template_Td_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_temperature="dust_gnilc/raw/gnilc_dust_small_scales_Td_cl_lmax16384_2023.06.06.fits.gz",
    freq_ref="353 GHz",
    seeds=[seed, seed + 1000, seed + 2000],
    max_nside=8192,
)

In [ ]:
sky = pysm3.Sky(nside=nside, component_objects=[dust])

In [ ]:
freq = 353 * u.GHz

In [ ]:
m = sky.get_emission(freq)

In [ ]:
hp.mollview(m[0], max=1e3)

### alm and cl calculation

In [ ]:
alm=hp.map2alm(m[0])
cl=pixell.curvedsky.alm2cl(alm)

In [ ]:
plt.plot(cl)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')

# Low resolution image

In [ ]:
nside_lowres = 128  # 
seed=5
#pysm simulation
dust_lowres = pysm3.ModifiedBlackBodyRealization(
    nside=nside_lowres,
    amplitude_modulation_temp_alm="dust_gnilc/raw/gnilc_dust_temperature_modulation_alms_lmax768.fits.gz",
    amplitude_modulation_pol_alm="dust_gnilc/raw/gnilc_dust_polarization_modulation_alms_lmax768.fits.gz",
    largescale_alm="dust_gnilc/raw/gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax1024_complex64.fits.gz",
    small_scale_cl="dust_gnilc/raw/gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz",
    largescale_alm_mbb_index="dust_gnilc/raw/gnilc_dust_largescale_template_beta_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_index="dust_gnilc/raw/gnilc_dust_small_scales_beta_cl_lmax16384_2023.06.06.fits.gz",
    largescale_alm_mbb_temperature="dust_gnilc/raw/gnilc_dust_largescale_template_Td_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_temperature="dust_gnilc/raw/gnilc_dust_small_scales_Td_cl_lmax16384_2023.06.06.fits.gz",
    freq_ref="353 GHz",
    seeds=[seed, seed + 1000, seed + 2000],
    max_nside=8192,
)

sky_lowres = pysm3.Sky(nside=nside_lowres, component_objects=[dust_lowres])

m_lowres = sky_lowres.get_emission(freq)

In [ ]:
alm_lowres=hp.map2alm(m_lowres[0])
cl_lowres=pixell.curvedsky.alm2cl(alm_lowres)

In [ ]:
plt.plot(cl,label='sky map')
plt.plot(cl_lowres,linestyle='--',label='sky map (lower resolution)')
#plt.plot(cl_car,label='2D map')
#plt.plot(cl_car_crop,label='2D map cropped')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')
plt.ylim(1e-3,1e6)
plt.legend()

## Convert sky map to 2D

In [ ]:
car_reso = (np.pi / np.round(np.pi / healpix_reso.value)) * u.radian
print(healpix_reso.to(u.arcmin), car_reso.to(u.arcmin))

In [ ]:
pixell.__version__

In [ ]:
m_car = pysm3.apply_smoothing_and_coord_transform(
    m, output_car_resol=car_reso, return_healpix=False, return_car=True
)

In [ ]:
m_car.shape

In [ ]:
plt.imshow(m_car[0,:,:], vmin=0, vmax=700, origin="lower")

In [ ]:
def get_cl_from_pixell_map(imap,lmax):
    # get an apodized mask, multiply the map before doing map2alm
    taper_mask = pixell.enmap.apod(pixell.enmap.ones(imap.shape, imap.wcs), width=100)
    alm_taper = pixell.curvedsky.map2alm(taper_mask * imap, lmax=lmax)

    # get the correction factor that accounts for the power lost due to only observing a
    # fraction of the sky
    # enmap.pixsizemap is a map of all the physical pixel areas in steradians
    w2 = np.sum(taper_mask.pixsizemap() * taper_mask**2) / (4*np.pi)

    #alm = pixell.curvedsky.map2alm(imap,lmax=lmax)
    # squaring and averaging over m is done by the alm2cl function
    cl=pixell.curvedsky.alm2cl(alm_taper)/w2
    return cl

## Calculate the cl from the 2D map

In [ ]:
lmax=3*nside-1
print('lmax is:',lmax)
print('nside is:',nside)

In [ ]:
#alm_car = pixell.curvedsky.map2alm(m_car,lmax=lmax)

In [ ]:
#reconstruct the sky image from the 2D alm
#hp.mollview(hp.alm2map(alm_car, nside=nside)[0], max=1e3)

In [ ]:
#calculate the cl from the 2D map
#cl_car=pixell.curvedsky.alm2cl(alm_car[0])
cl_car=get_cl_from_pixell_map(m_car[0],lmax)

In [ ]:
plt.plot(cl,label='sky map')
plt.plot(cl_car,label='2D map')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')
plt.ylim(1e-3,1e6)
plt.legend()

## Crop image to a a square centered in the middle

In [ ]:
img_size=512
start_x=int(len(m_car[0,0,:])/2-img_size/2)#500
start_y=int(len(m_car[0,:,0])/2-img_size/2)#
print('The image starts at x',start_x,'and y ',start_y)

In [ ]:
crop_image=m_car[0,start_y:start_y+img_size,start_x:start_x+img_size]

In [ ]:
plt.imshow(crop_image, vmin=0, vmax=700, origin="lower")

In [ ]:
#alm_car_crop = pixell.curvedsky.map2alm(crop_image,lmax=lmax)
cl_car_crop=get_cl_from_pixell_map(crop_image,lmax)#pixell.curvedsky.alm2cl(alm_car_crop)

In [ ]:
plt.plot(cl,label='sky map')
plt.plot(cl_car,label='2D map')
plt.plot(cl_car_crop,label='2D map cropped')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')
plt.ylim(1e-3,1e6)
plt.legend()

### Save image to file

In [ ]:
crop_image.shape

In [ ]:
#saving this way (as a png) will lose the wcs
fig = plt.figure(frameon=False)
plt.imshow(crop_image, vmin=0, vmax=1000, origin="lower")
plt.axis('off')
#fig.savefig('out.png', bbox_inches='tight', pad_inches=0)
plt.imsave(fname='out.png', arr=crop_image, vmin=0, vmax=1000, origin="lower", format='png') #imsave save the array "as is"

In [ ]:
#save the map (and wcs)
file_map_name='amap.hdf'
pixell.enmap.write_map(file_map_name,crop_image,fmt='hdf')

## Read the data from the file

In [ ]:
#this can read the image but wcs cannot be read from a png
img = imread('out.png')
plt.imshow(img)

### Read the map from the HDF file (includes a wcs)

In [ ]:
read_map= pixell.enmap.read_map(file_map_name,fmt='hdf')

In [ ]:
#calculate the cl from the read image
cl_car_img_file=get_cl_from_pixell_map(read_map,lmax)#pixell.curvedsky.alm2cl(pixell.curvedsky.map2alm(img,lmax=lmax))

In [ ]:
plt.plot(cl,label='sky map')
plt.plot(cl_car,label='2D map')
plt.plot(cl_car_crop,label='2D map cropped')
plt.plot(cl_car_img_file,label='2D map cropped (from file)')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('l')
plt.ylabel('Spectrum')
plt.title(r'$c_l$ spectrum')
plt.ylim(1e-3,1e6)
plt.legend()

### Output monochromatic figure

In [ ]:
#scale the array to 0-255 (need it to create the image with pillow uint8)
img_scaled = (crop_image /crop_image.max()) * 255
img_scaled= img_scaled.astype(np.uint8)

In [ ]:
#create the pillow image from the array
img_pil=Image.fromarray(img_scaled)
#flip the image as we did with the 'origin=lower' in pyplot
flip_img=img_pil.transpose(Image.FLIP_TOP_BOTTOM)
#write the image monochromatic in a png file
flip_img.save('out_pillow.png')

In [ ]:
flip_img

In [ ]:
#read the pillow image
img_read = imread('out_pillow.png')
plt.imshow(img_read,vmin=0,vmax=0.05)
plt.colorbar()